---

# 231004
## 목표
1. 실행 시간 2초 이내가 되도록

In [2]:
import os

directory = "./data"

all_items = sorted(os.listdir(directory))

In [3]:
"""
목표함수 (1)
해당 목적함수는 결정변수 theta_wt와 요격확률과 타깃의 weight의 곱의 합을 최대화함으로써 요격확률이 높을 때 표적-발사대 쌍의 발사 시점에 무기할당을 하는 의미로 해석할 수 있다.
input은, 할당여부를 나타내는 theta_wt, 요격확률 PK_wt, 타깃의 weight
"""
def objective_fun_1(theta_wt, weight, PK_wt):
    res = 0
    for t in range(T):
        
        a = 1
        for w in range(W):
            for m in range(M):
                p = 1 - PK_wt[w,t] * theta_wt[w,t,m]
                a *= p
        
        a *= weight[t]

        # print(f"타깃 {t}에 대한 성공 값 : {a}")
        res += a


    return res


In [4]:
import time
import numpy as np
import time

In [5]:
data_list = []
for data in all_items:
    path = os.path.join(directory,data)

    start_time = time.time()
    # # 데이터의 path
    # path = "data/wta_100x200x3.txt"

    # reading data from text file
    f = open(path, "r")
    lines = f.readlines()
    f.close()

    # reading first line
    # FN_T 는 타깃 당 최대 미사일 할당량
    W, T, M_W = [int(v) for v in lines[0].split()]
    # asssuming all weapons have the same availability:
    M_W = [M_W] * W

    M = sum(M_W)

    # PK_wt 배열 초기화
    PK_wt = np.zeros((W, T))

    # reading target weights, len(weight) == T
    weight=[]
    for line in lines[1:T + 1]:
        weight.append(int(line))

    # reading probabilities
    for line in lines[T + 1:]:
        splitted = line.split()
        w, t, p = int(splitted[0]), int(splitted[1]), float(splitted[2])
        PK_wt[w][t] = p

    import copy
    # 할당 여부를 나타내는 theta_wt
    theta_wt = np.zeros((W, T, M), dtype=int)

    # 발사대에 적재된 유도탄 수량 리스트 left_M_W으로 복사
    left_M_W = copy.copy(M_W)

    # 행렬의 원소들을 1차원 배열로 변환, 이때, 타깃 가중치 추가
    flattened = (PK_wt*weight).flatten()

    # 원소중 가장 큰 값 인덱스 저장
    max_index = np.argmax(flattened)

    i = 0
    m = 0

    indices_list = []

    while True :
        if i == T * W :
            break
        
        # 가장 요격확률 * 가중치가 큰 값
        v = max_index
        
        row = v // T # Weapon
        col = v % T # Target
        # print(f"Value: {flattened[v]}, Index: (Weapon : {row}, Target : {col})")
        
        # 해당 Weapon의 적재된 유도탄이 남아 있다면,
        if left_M_W[row] > 0:
        
            # 할당할 해당 무기(row)의 유도탄 개수 감소
            left_M_W[row] -= 1

            # 해당 W T M에 1 부여
            theta_wt[row][col][m] = 1

            # 무기가 할당된 indices_list에 추가
            indices_list.append(v)

            # 다음 미사일 인덱스로 이동
            m += 1

            weight[col] *= (1-PK_wt[row][col])

            # 행렬의 원소들을 1차원 배열로 변환, 이때, 타깃 가중치 추가
            flattened = (PK_wt*weight).flatten()

            max_index = np.argmax(flattened)

        
        else :
            # 미사일 소진 시 요격확률 0으로 수정
            # print(f"not enough Missile on Weapon {row}")
            PK_wt[row] = np.zeros_like(col)
            
        
        # 할당할 수 있는 유도탄의 개수를 모두 소모했다면 종료.
        if sum(left_M_W) == 0:
            break
        
        # 다음으로 요격확률이 높은 인덱스로 이동
        # 행렬의 원소들을 1차원 배열로 변환, 이때, 타깃 가중치 추가
        flattened = (PK_wt*weight).flatten()

        max_index = np.argmax(flattened)
        # 남은 미사일 개수 출력
        # print(sum(left_M_W))
    end_time = time.time()
    ov = objective_fun_1(theta_wt,weight,PK_wt)

    elapsed_time = end_time - start_time
    print(f"data : {data} | objective vlaue : {ov:.4f} | Elapsed time: {elapsed_time:.2f} seconds")
    data_list.append([data,ov,elapsed_time])

data : wta_100x200x1.txt | objective vlaue : 2926.7789 | Elapsed time: 0.04 seconds
data : wta_100x200x2.txt | objective vlaue : 127.8051 | Elapsed time: 0.04 seconds
data : wta_100x200x3.txt | objective vlaue : 17.0246 | Elapsed time: 0.05 seconds
data : wta_150x300x1.txt | objective vlaue : 3759.0585 | Elapsed time: 0.08 seconds
data : wta_150x300x2.txt | objective vlaue : 121.4405 | Elapsed time: 0.11 seconds
data : wta_150x300x3.txt | objective vlaue : 13.0685 | Elapsed time: 0.14 seconds
data : wta_200x400x1.txt | objective vlaue : 5042.4856 | Elapsed time: 0.18 seconds
data : wta_200x400x2.txt | objective vlaue : 141.2811 | Elapsed time: 0.25 seconds
data : wta_200x400x3.txt | objective vlaue : 12.0670 | Elapsed time: 0.31 seconds
data : wta_250x500x1.txt | objective vlaue : 6801.2688 | Elapsed time: 0.33 seconds
data : wta_250x500x2.txt | objective vlaue : 139.0970 | Elapsed time: 0.45 seconds
data : wta_250x500x3.txt | objective vlaue : 12.1924 | Elapsed time: 0.58 seconds
data

In [6]:
#CSV 파일로 저장
import csv

filename = '231004_GRDsol.csv'
header = ['Data','Object Value','Runtime']

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data_list)

In [7]:
# theta_wt에서 3번째 축을 따라 합산
sol = np.sum(theta_wt, axis=2)

# 0이 아닌 원소의 인덱스 찾기
non_zero_indices = np.argwhere(sol != 0)

# 인덱스 출력
for index in non_zero_indices:
    temp_w, temp_t = tuple(index)
    print(f"x({temp_w},{temp_t})={sol[temp_w][temp_t]}")

x(0,31)=1
x(0,37)=1
x(0,59)=1
x(1,12)=1
x(1,75)=1
x(1,95)=1
x(2,19)=1
x(2,41)=1
x(2,65)=1
x(3,22)=1
x(3,39)=1
x(3,87)=1
x(4,0)=1
x(4,43)=1
x(4,86)=1
x(5,17)=1
x(5,55)=2
x(6,45)=1
x(6,61)=1
x(6,71)=1
x(7,7)=1
x(7,58)=1
x(7,64)=1
x(8,90)=2
x(8,92)=1
x(9,18)=1
x(9,33)=1
x(9,66)=1
x(10,41)=1
x(10,47)=1
x(10,54)=1
x(11,32)=1
x(11,46)=1
x(11,99)=1
x(12,25)=1
x(12,27)=1
x(12,68)=1
x(13,13)=1
x(13,45)=1
x(13,62)=1
x(14,79)=1
x(14,85)=1
x(14,97)=1
x(15,4)=1
x(15,18)=1
x(15,46)=1
x(16,31)=1
x(16,78)=1
x(16,93)=1
x(17,34)=1
x(17,77)=1
x(17,96)=1
x(18,36)=1
x(18,53)=1
x(18,57)=1
x(19,14)=1
x(19,73)=1
x(19,83)=1
x(20,16)=1
x(20,40)=1
x(20,89)=1
x(21,67)=2
x(21,91)=1
x(22,17)=1
x(22,57)=1
x(22,94)=1
x(23,52)=2
x(23,69)=1
x(24,51)=1
x(24,61)=1
x(24,80)=1
x(25,59)=1
x(25,88)=1
x(25,93)=1
x(26,1)=1
x(26,63)=1
x(26,64)=1
x(27,32)=1
x(27,48)=1
x(27,49)=1
x(28,23)=2
x(28,85)=1
x(29,11)=1
x(29,38)=1
x(29,74)=1
x(30,21)=2
x(30,42)=1
x(31,20)=1
x(31,22)=1
x(31,28)=1
x(32,6)=1
x(32,37)=1
x(32,76)=1
x(33,10)=2